In [1]:
%%capture
pip install cobra

In [2]:
%%capture
pip install cameo

In [3]:
import cobra
import os
import pandas as pd
import cameo

In [4]:
os.chdir("/content/drive/MyDrive/Halomonas_Model_devt")

In [ ]:
%%capture
model= cobra.io.read_sbml_model("/content/drive/MyDrive/Halomonas_Model_devt/Complete_HalomonasPolished.xml") ##this model still needs to be edited with Libsbml

In [ ]:
#model= cobra.io.read_sbml_model("/content/drive/MyDrive/Halomonas_Model_devt/HalomonasProkkamodelV1.xml")

In [ ]:
model

Name,Halomonas_RAST
Memory address,0x07fa8e4a2b790
Number of metabolites,1531
Number of reactions,2306
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, extracellular space, periplasm"


In [ ]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,5.431,4,40.04%
bz_e,EX_bz_e,9.7E-05,7,0.00%
ca2_e,EX_ca2_e,0.005049,0,0.00%
cl_e,EX_cl_e,0.005049,0,0.00%
cobalt2_e,EX_cobalt2_e,9.7E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006877,0,0.00%
fe3_e,EX_fe3_e,0.01409,0,0.00%
fru_e,EX_fru_e,3.922,6,43.37%
glc__D_e,EX_glc__D_e,1.5,6,16.59%
k_e,EX_k_e,0.1893,0,0.00%


In [ ]:
model.medium

{'EX_asn__L_e': 1000.0,
 'EX_bz_e': 9.7e-05,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_fru_e': 10.0,
 'EX_glc__D_e': 1.5,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [ ]:
medium = model.medium
medium["EX_fru_e"] = 0.0
medium["EX_glc__D_e"] = 10.0
#medium["EX_bz_e"] = 0.0
medium["EX_asn__L_e"] = 3
model.medium = medium

In [ ]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,3,4,39.90%
bz_e,EX_bz_e,5.358E-05,7,0.00%
ca2_e,EX_ca2_e,0.002789,0,0.00%
cl_e,EX_cl_e,0.002789,0,0.00%
cobalt2_e,EX_cobalt2_e,5.358E-05,0,0.00%
cu2_e,EX_cu2_e,0.0003799,0,0.00%
fe3_e,EX_fe3_e,0.007782,0,0.00%
glc__D_e,EX_glc__D_e,3.012,6,60.09%
k_e,EX_k_e,0.1046,0,0.00%
mg2_e,EX_mg2_e,0.004648,0,0.00%


#Strain optimization with Cameo

In [ ]:
from cameo import fba
%time fba_result = fba(model)

CPU times: user 199 ms, sys: 9.65 ms, total: 209 ms
Wall time: 306 ms


In [ ]:
fba_result.data_frame

,flux
14GLUCANabcpp,0.00000
14GLUCANtexi,0.00000
1PPDCRc,0.00000
2AGPE120tipp,0.00000
2AGPE140tipp,0.00000
...,...
EutB,0.00000
ASPSALy,-16.42656
ECTOtex,0.00000
ECTOabcpp,0.00000


In [ ]:
#Pfba
from cameo import pfba
%time pfba_result = pfba(model)

CPU times: user 583 ms, sys: 6.81 ms, total: 589 ms
Wall time: 722 ms


In [ ]:
pfba_result.objective_value

230.27199076208726

In [ ]:
abs(fba_result.data_frame.flux).sum()

320.31671194875565

In [ ]:
from cameo import flux_variability_analysis

In [ ]:
fva_result = flux_variability_analysis(model)
fva_result.data_frame

,lower_bound,upper_bound
14GLUCANabcpp,0.0,0.000000
14GLUCANtexi,0.0,0.000000
1PPDCRc,0.0,23.000145
2AGPE120tipp,0.0,258.150388
2AGPE140tipp,0.0,258.150388
...,...,...
EutB,-1000.0,1000.000000
ASPSALy,-1000.0,290.751940
ECTOtex,0.0,0.000000
ECTOabcpp,-3.0,0.000000


In [ ]:
from cameo.visualization import plotting

In [ ]:
fva_result2 = flux_variability_analysis(model,fraction_of_optimum=0.5)
fva_result2.data_frame

,lower_bound,upper_bound
14GLUCANabcpp,0.0,0.000000
14GLUCANtexi,0.0,0.000000
1PPDCRc,0.0,19.198547
2AGPE120tipp,0.0,244.613162
2AGPE140tipp,0.0,244.613162
...,...,...
EutB,-1000.0,1000.000000
ASPSALy,-1000.0,223.065808
ECTOtex,0.0,0.000000
ECTOabcpp,-1.5,0.000000


In [ ]:
from cameo import phenotypic_phase_plane

In [ ]:
#model.reactions.EX_o2_e.lower_bound = -10
result = phenotypic_phase_plane(model,
                                variables=[model.reactions.Growth],
                                objective=model.reactions.PHBS_syn_1,
                                points=10)

In [ ]:
#result.plot()
result.data_frame

,Growth,objective_lower_bound,objective_upper_bound,c_yield_lower_bound,c_yield_upper_bound,mass_yield_lower_bound,mass_yield_upper_bound
0,0.000000,0.0,15.000102,0.0,6.250043,NaN,NaN
1,0.061480,0.0,14.439887,0.0,6.016619,NaN,NaN
2,0.122960,0.0,13.879671,0.0,5.783196,NaN,NaN
3,0.184441,0.0,13.319455,0.0,5.549773,NaN,NaN
4,0.245921,0.0,12.759239,0.0,5.316350,NaN,NaN
5,0.307401,0.0,12.199023,0.0,5.082926,NaN,NaN
6,0.368881,0.0,11.638808,0.0,4.849503,NaN,NaN
7,0.430361,0.0,11.078592,0.0,4.616080,NaN,NaN
8,0.491841,0.0,10.518376,0.0,4.382657,NaN,NaN
9,0.553322,0.0,9.958160,0.0,4.149233,NaN,NaN


In [ ]:
result.data_frame.to_excel("phenotypic_phase_planes.xlsx")

In [ ]:
from cameo.flux_analysis.analysis import flux_balance_impact_degree

In [ ]:
%time fbid = flux_balance_impact_degree(model, ["PHBS_syn_1"])

CPU times: user 1min 50s, sys: 477 ms, total: 1min 51s
Wall time: 1min 56s


In [ ]:
fbid

,perturbed
1PPDCRc,False
2AGPE140tipp,False
2AGPE160tipp,False
2AGPE161tipp,True
2AGPE180tipp,False
...,...
EutC,False
EutB,False
ASPSALy,False
ECTOabcpp,False


#Gene Knockout strategies

In [ ]:
#optgene
from cameo.strain_design.heuristic.evolutionary_based import OptGene

In [ ]:
optgene = OptGene(model)

In [ ]:
result = optgene.run(target="PHBS_syn_1",
                     biomass="Growth",
                     substrate="glc__D_e",
                     max_evaluations=5000,
                     plot=False)

Starting optimization at Thu, 14 Apr 2022 07:17:07


HBox()

Finished after 00:17:40


In [ ]:
result

No solutions found


,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness


In [ ]:
#optKnock
from cameo.strain_design.deterministic.linear_programming import OptKnock

In [ ]:
optknock = OptKnock(model, fraction_of_optimum=0.1)

/usr/local/lib/python3.7/dist-packages/cameo/strain_design/deterministic/linear_programming.py:119: UserWarning: You are trying to run OptKnock with glpk_interface. This might not end well.
  self._model.solver.interface.__name__.split(".")[-1])


In [ ]:
result = optknock.run(max_knockouts=2, target="PHBS_syn_1", biomass="Growth")

In [ ]:
result

HBox()

,reactions,size,PHBS_syn_1,biomass,fva_min,fva_max
0,"{ORNCD, P5CR}",2,14.495908,0.055332,0.0,15.000102


In [ ]:
model.reactions.ORNCD

Reaction identifier,ORNCD
Name,Ornithine cyclodeaminase
Memory address,0x07fa8c65017d0
Stoichiometry,orn_c --> nh4_c + pro__L_c Ornithine --> Ammonium + L-Proline
GPR,fig_6666666_879901_peg_2668 or fig_6666666_879901_peg_933
Lower bound,0.0
Upper bound,1000.0


In [ ]:
model.reactions.P5CR

Reaction identifier,P5CR
Name,Pyrroline-5-carboxylate reductase
Memory address,0x07fa8c650f790
Stoichiometry,1pyr5c_c + 2.0 h_c + nadph_c --> nadp_c + pro__L_c 1-Pyrroline-5-carboxylate + 2.0 H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> Nicotinamide adenine dinucleotide phosphate + L-Proline
GPR,fig_6666666_879901_peg_3180
Lower bound,0.0
Upper bound,1000.0


In [5]:
!git add .

In [6]:
!git commit -m  "Added a colab notebook with initial PHB optimization simulations with Cameo"


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@63f13699124b.(none)')


In [7]:
!git config --global user.email "enuhblaise@yahoo.com"


In [8]:
!git config --global user.name "EnuhBlaise"

In [9]:
!git push

Everything up-to-date


In [10]:
pwd

'/content/drive/MyDrive/Halomonas_Model_devt'

In [11]:
ls

 BIGG_universal_model.json                     HalomonasmodelV1.json
 CarbonSimulations.csv                         HalomonasmodelV1.xml
'CarveMe Models'/                              HalomonasmodelV2.xml
 Complete_Halomonas.json                       HalomonasmodelV3.json
 Complete_HalomonasPolished.xml                HalomonasmodelV3.xml
 Complete_Halomonas.xml                        HalomonasProkkamodelV1.xml
 consistent_model.xml                          Halomonas_RASTV1.xml
 data/                                         Halomonas_RASTV3.json
'(+DemandPHBrxns)_Model.xml'                   Halomonas_RASTV3.xml
'extract reactions'/                           Halomonas_RAST.xml
 FVA.xlsx                                      Notebooks/
'Genome Scale Model Reconstruction Process'/   phenotypic_phase_planes.xlsx
 Halomonas_annotation_feature_dna_Model.xml    ProkkaHalomonasModel.xml
 Halomonas_COBRAModel.xml                      Readme.txt
 HalomonasModelFBC.xml                         un

In [12]:
cd Notebooks

/content/drive/MyDrive/Halomonas_Model_devt/Notebooks


In [13]:
ls

 Halomonas_Add_missing_PHB_Reactions.ipynb
'Halomonas_Add_PHB_reactions and Metabolites.ipynb'
'Halomonas_ Add_reactions_Ectoine_Synthesis.ipynb'
 Halomonas_CarbonSource_Simulations.ipynb
 Halomonas_Choline_Betaine.ipynb
 Halomonas_GEM_Min_Media.ipynb
 Halomonas_Gene_deletion_simulations.ipynb
'Halomonas_SettingSBO terms.ipynb'
'Halomonas_Simulating PHB production with different carbon sources.ipynb'
 Random_tests.ipynb
'Strain Optimization.ipynb'


In [15]:
!git push origin master

Everything up-to-date


In [16]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   Random_tests.ipynb
	new file:   Strain Optimization.ipynb
	new file:   ../phenotypic_phase_planes.xlsx

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Strain Optimization.ipynb



In [ ]:
!git add 